# 跳跃收益因子

## 导入模块

In [1]:
import numpy as np
import pandas as pd
import feather
import os
from tqdm.notebook import tqdm

## 读入数据

In [2]:
jump = feather.read_dataframe('../data/jump/jump.feather')

### 交易日历

In [3]:
trade_date = jump['date'].sort_values().unique()
start_date = '2019-01-01'
end_date = '2024-05-31'
adj_date = trade_date[(trade_date >= start_date) & (trade_date <= end_date)]
calc_start_date = np.array([], dtype=np.datetime64)

for ad in adj_date:
    trade_date_before = trade_date[trade_date <= ad]
    if (len(trade_date_before) > 20):
        csd = trade_date_before[-20]
    else:
        csd = trade_date_before[0]
    calc_start_date = np.append(calc_start_date, csd)

## 跳跃收益因子

In [4]:
ret_jump = None
tqdm_date = tqdm(zip(adj_date, calc_start_date), total=len(adj_date))
for ad, csd in tqdm_date:
    jump_date = jump[(jump['date'] <= ad) & (jump['date'] >= csd)]
    ret_jump_date = jump_date.groupby('issue')['ret_jump'].sum().reset_index()
    ret_jump_date['date'] = ad
    ret_jump = pd.concat([ret_jump, ret_jump_date])
    del jump_date, ret_jump_date
os.makedirs('../data/ret_jump_daily/', exist_ok=True)
feather.write_dataframe(ret_jump, '../data/ret_jump_daily/ret_jump.feather')

  0%|          | 0/1312 [00:00<?, ?it/s]